In [1]:
#get_seq_from_file function reads in a FASTA file and returns a list of tuples, where each tuple contains the header and DNA 
#sequence for a single sequence in the file

def get_seq_from_file(fname):
    with open(fname, 'r') as file:

        sequences = []  # An empty list to hold the headers and DNA sequences in the FASTA file
        header = ''# A variable to hold the current header
        dna = '' # A variable to hold the current DNA sequence

        for line in file:
            line = line.strip()
            if line.startswith('>'): # If the line starts with '>'
                if dna:
                    sequences.append((header, dna))
                    dna = ''
                header = line[1:]
            else:
                dna += line.upper()
        if dna:
            sequences.append((header, dna))
    return sequences

#function takes a DNA sequence as input and returns a list of codons, which are substrings of the DNA sequence of length 3. 

def get_codons(seq_frame):
    codons = []
    for i in range(0, len(seq_frame), 3):
        if len(seq_frame[i:i+3]) == 3:
            codons.append(seq_frame[i:i+3])
    return codons

#function takes a DNA sequence as input and returns its reverse complement
def reverse_complement(seq: str) -> str:    #Shatakshi Shewale
    # Define a dictionary that maps each nucleotide base to its complement base
    complement_dict = {"A": "T", "T": "A", "C": "G", "G": "C"}   
    
    # Create a new string by iterating over the reversed input sequence and looking up each base in the complement dictionary
    reverse_complement_seq = "".join(complement_dict.get(base, "") for base in reversed(seq))
    
    # Return the reverse complement sequence
    return reverse_complement_seq    

#function takes a DNA sequence as input and returns six possible codon frames
def get_seq_frames(dna):    #Shatakshi Shewale
    # Reverse complement the DNA sequence
    dna_rev_comp = reverse_complement(dna)

    # Create an empty list to store the different reading frames
    seq_frames = []

    # Add the original DNA sequence as the first reading frame
    seq_frames.append(dna)

    # Add the second reading frame, starting from the second nucleotide
    seq_frames.append(dna[1:])

    # Add the third reading frame, starting from the third nucleotide
    seq_frames.append(dna[2:])

    # Add the reverse complement of the DNA sequence as the fourth reading frame
    seq_frames.append(dna_rev_comp)

    # Add the fifth reading frame of the reverse complement, starting from the second nucleotide
    seq_frames.append(dna_rev_comp[1:])

    # Add the sixth reading frame of the reverse complement, starting from the third nucleotide
    seq_frames.append(dna_rev_comp[2:])

    # Return the list of reading frames
    return seq_frames


# function takes a list of codon frames and a minimum length and identifies all possible ORFs in the 
#codon frames that are longer than the minimum length

def get_orfs(codon_frames_all, min_length):
    frame_orfs = [] # stores information about identified ORFs in each frame
    orfs = [] #stores the actual ORF sequences
    
    #Iteration over each codon frame
    for fi, codon_frame in enumerate(codon_frames_all):
        start_codon = 'ATG'
        end_codons = ['TAA', 'TAG', 'TGA']
        
        sc_flag = False # initialize start codon flag to False
        ec_flag = False # initialize end codon flag to False 
        sc_idx = None # initialize start codon index to None
        ec_idx = None # initialize end codon index to None
        orf_flag = False # initialize ORF flag to False
        
    #     print(codon_frame)
    
       # iterate over each codon in the current frame
        for idx,codon in enumerate(codon_frame):
            # if an ORF has not yet been identified
            if not orf_flag:
                # if a start codon has not yet been identified
                if not sc_flag:
                    # if the current codon is a start codon
                    if codon == start_codon:
                        sc_idx = idx
                        sc_flag = True
        #                 print(f'SC IDX: {sc_idx}, {codon}')
                else:
                    for i in range(sc_idx+1, len(codon_frame)):
                        codon = codon_frame[i]
                        if codon in end_codons:
                            ec_idx = i
                            ec_flag = True
        #                     print(f'EC IDX: {ec_idx}, {codon}')

                        if sc_flag and ec_flag:
                            orf_len = (ec_idx+1-sc_idx)*3
                            if orf_len >= min_length:
                                print(f'ORF Length: {orf_len}')
                                print(f'Frame {fi+1}={codon_frame}')
                                print(f'SC IDX: {sc_idx}, {codon}')
                                print(f'EC IDX: {ec_idx}, {codon}')
                                orf = codon_frame[sc_idx:ec_idx+1]
                                orfs.append(orf)
                                frame_orfs.append((fi+1, sc_idx, orf_len, orf))
                                orf_flag=True
                            sc_flag = False
                            ec_flag = False
                            sc_idx = None
                            ec_idx = None
                    sc_flag = False
                    ec_flag = False
                    sc_idx = None
                    ec_idx = None
                    
    return frame_orfs


def write_output(header, frame_orfs):
    for frame_orf in frame_orfs:
        fi = frame_orf[0]
        sc_idx = frame_orf[1]
        orf_len = frame_orf[2]
        orf = frame_orf[3]
        if fi < 3:
            pos = sc_idx*3+1
        else:
            pos = -(sc_idx+1)*3
        print(f'>{header} | FRAME = {fi} POS = {pos} LEN = {orf_len}')
        for i in range(0,len(orf),15):
            print(' '.join(orf[i:i+15]))
            
            
def run_all():
    fname = input('Enter FASTA file:')
    min_len = int(input('Enter minimum length in bp for ORFs:'))
    
    sequences = get_seq_from_file(fname)
    with open('code_output.txt', 'w') as f:
        f.write('This program will find ORFs for sequences. When prompted, please enter a FASTA file with 1 or more sequences.')
        f.write('\n\n')
        f.write(f'Enter FASTA file: {fname}\n\n')
        f.write(f'Enter minimum length in bp for ORFs: {min_len}\n')
        
        
        for sequence in sequences:
            header = sequence[0]
            dna = sequence[1]
            print(header)

            seq_frames = get_seq_frames(dna)
            codon_frames = [get_codons(seq_frame) for seq_frame in seq_frames]
            frame_orfs = get_orfs(codon_frames, min_len)

            for frame_orf in frame_orfs:
                fi = frame_orf[0]
                sc_idx = frame_orf[1]
                orf_len = frame_orf[2]
                orf = frame_orf[3]
                if fi < 3:
                    pos = sc_idx*3+1
                else:
                    pos = -(sc_idx+1)*3
                f.write(f'>{header} | FRAME = {fi} POS = {pos} LEN = {orf_len}')
                f.write('\n')
                for i in range(0,len(orf),15):
                    f.write(' '.join(orf[i:i+15]))
                    f.write('\n')

run_all()

Enter FASTA file:sequence.fasta
Enter minimum length in bp for ORFs:300
Test1
ORF Length: 996
Frame 1=['GAG', 'CAA', 'GAG', 'TGA', 'AGA', 'GCG', 'AAG', 'AGA', 'ATG', 'GCT', 'CCC', 'AAG', 'GGT', 'TTA', 'ATC', 'TTT', 'TTG', 'GCT', 'GTG', 'TTA', 'TGC', 'TTC', 'TCA', 'GCA', 'CTG', 'TCA', 'CTG', 'AGT', 'CGT', 'TGT', 'CTT', 'GCG', 'GAG', 'GAT', 'AAT', 'GGA', 'CTT', 'GTT', 'ATG', 'AAC', 'TTC', 'TAC', 'AAG', 'GAA', 'TCA', 'TGC', 'CCT', 'CAG', 'GCT', 'GAA', 'GAC', 'ATC', 'ATC', 'AAA', 'GAA', 'CAA', 'GTC', 'AAG', 'CTT', 'CTC', 'TAC', 'AAG', 'CGC', 'CAC', 'AAG', 'AAC', 'ACT', 'GCT', 'TTC', 'TCC', 'TGG', 'CTC', 'AGA', 'AAC', 'ATC', 'TTC', 'CAT', 'GAC', 'TGT', 'GCT', 'GTT', 'CAG', 'AGT', 'TGT', 'GAT', 'GCT', 'TCA', 'CTG', 'TTG', 'CTG', 'GAC', 'TCC', 'ACA', 'AGA', 'AGG', 'AGC', 'TTG', 'TCT', 'GAG', 'AAG', 'GAA', 'ACA', 'GAT', 'AGA', 'AGC', 'TTT', 'GGG', 'TTG', 'AGA', 'AAT', 'TTC', 'AGG', 'TAC', 'ATT', 'GAG', 'ACC', 'ATC', 'AAA', 'GAA', 'GCT', 'TTG', 'GAA', 'AGG', 'GAA', 'TGC', 'CCA', 'GGA', 'GTT', '